In [6]:
import numpy as np

将评分矩阵 ${A}^{T}$ 映射到低维空间 $A^{T}U_{m*k}\sum_{k*k}^{-1}$

In [13]:
def loadExData():
    return [[0,0,1,0,0,2,0,0,0,0,5],
           [0,0,0,5,0,3,0,0,0,0,3],
           [0,0,0,0,4,1,0,1,0,4,0],
           [3,3,4,0,0,0,0,2,2,0,0],
           [5,4,2,0,0,0,0,5,5,0,0],
           [0,0,0,0,5,0,1,0,0,0,0],
           [4,1,4,0,0,0,0,4,5,0,1],
           [0,0,0,4,0,4,0,0,0,0,4],
           [0,0,0,2,0,2,5,0,0,1,2],
           [1,0,0,4,0,0,0,1,2,0,0]]

def CosSimilarCompute(U_k, W_t):
    if( len(U_k) != len(W_t) ):
        assert("user vector size not equals item vector ")
    if(len(U_k)==0):
        assert("user vector size  equal to 0")
    i = 0
    Score1 = 0
    Score2 = 0
    Score3 = 0
    while( i < len(U_k) ):
        Score1 = Score1 + U_k[i]*W_t[i]
        Score2 = Score2 + U_k[i]*U_k[i]
        Score3 = Score3 + W_t[i]*W_t[i]
        i = i+1
    if(Score3==0 or Score2==0):
        assert("user or item vector equal to 0")
    return Score1*1.0/(math.sqrt(Score2)*math.sqrt(Score3))



def cosSim(U_k, W_t):
    num = float(U_k.T*W_t)
    denom = np.linalg.norm(U_k)*np.linalg.norm(W_t)
    return 0.5+0.5*(num/denom)

# simMeas相似度计算函数
def svdEst(userData,xformedItems, user, simMeas, item) :
    n = np.shape(xformedItems)[0]
    simTotal = 0.0; ratSimTotal = 0.0
    # 对于给定的用户，for循环所有物品，计算与item的相似度
    for j in range(n) :
        userRating = userData[:,j]
        if userRating == 0 or j == item : continue
        similarity = simMeas(xformedItems[item, :].T, xformedItems[j, :].T)
        # print便于了解相似度计算的进展情况
        print ('the %d and %d similarity is : %f' % (item, j, similarity))
        # 对相似度求和
        simTotal += similarity
        # 对相似度及评分值的乘积求和
        ratSimTotal += similarity * userRating
    if simTotal == 0 : return 0
    else : return ratSimTotal/simTotal
    
    
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=svdEst):
    U,Sigma,VT = np.linalg.svd(dataMat)
    # 使用奇异值构建一个对角矩阵
    Sig4 = np.mat(np.eye(4)*Sigma[:4])
    # 利用U矩阵将物品转换到低维空间中
    xformedItems = dataMat.T * U[:, :4] * Sig4.I
    print('xformedItems=',xformedItems)
    print('xformedItems行和列数', np.shape(xformedItems))
    
    unratedItems = np.nonzero(dataMat[user, :].A == 0)[1]
    print('dataMat[user, :].A=',dataMat[user, :].A)
    print('np.nonzero(dataMat[user, :].A == 0)结果为', np.nonzero(dataMat[user, :].A == 0))
    # 如果不存在未评分物品，退出函数，否则在所有未评分物品上进行循环
    if len(unratedItems) == 0: return ('you rated everything')
    itemScores = []
    for item in unratedItems:
        print('item=',item)
   # 对于每个未评分物品，通过调用standEst()来产生该物品基于相似度的预测评分。
        estimatedScore = estMethod(dataMat[user, :],xformedItems, user, simMeas, item)
        # 该物品的编号和估计得分值会放在一个元素列表itemScores
        itemScores.append((item, estimatedScore))
    # 寻找前N个未评级物品
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

In [14]:
myMat=np.mat(loadExData())
result=recommend(myMat, 1, estMethod=svdEst) # 为user1推荐

xformedItems= [[-4.94993155e-01 -8.55622175e-02 -2.48611806e-02  3.33301232e-02]
 [-3.13458547e-01 -7.52514984e-02 -7.48135856e-03 -3.50541410e-02]
 [-3.78450772e-01 -4.30110266e-02 -2.73248525e-02 -3.16127753e-01]
 [-9.66651163e-02  5.96631453e-01 -1.11181970e-01  7.08108827e-01]
 [-1.48606103e-02  4.20778007e-02  8.58513789e-01  4.28827005e-02]
 [-5.34500615e-02  4.98695994e-01  4.12721285e-02 -1.51444295e-01]
 [-1.53015570e-02  1.73637176e-01  1.96700179e-01 -5.96048540e-02]
 [-4.69111042e-01 -7.14523528e-02  8.17335025e-02  6.74284466e-02]
 [-5.18848182e-01 -6.64352384e-02 -3.46135811e-02  1.62309665e-01]
 [-1.56328052e-02  6.08803417e-02  4.39273573e-01  4.45686018e-04]
 [-1.06449559e-01  5.79097298e-01 -8.90946777e-02 -5.80592096e-01]]
xformedItems行和列数 (11, 4)
dataMat[user, :].A= [[0 0 0 5 0 3 0 0 0 0 3]]
np.nonzero(dataMat[user, :].A == 0)结果为 (array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 1, 2, 4, 6, 7, 8, 9]))
item= 0
the 0 and 3 similarity is : 0.524508
the 0 and 5 similarity is 

In [15]:
print(result)

[(8, matrix([[3.83996333]])), (7, matrix([[3.74479602]])), (0, matrix([[3.71568381]]))]


In [16]:
U, Sigma, VT = np.linalg.svd(myMat)
new_data = U[:,:4] * np.mat(np.eye(4) * Sigma[:4]) * VT[:4,:] # k = 4

In [19]:
print(new_data)

[[ 0.06661544  0.15564622  1.34358251 -0.05523537 -0.30895421  2.49158087
   0.81721484 -0.06709356 -0.28593042  0.07729121  4.41737223]
 [ 0.00698734 -0.20783121 -0.30882512  4.83183689 -0.26688288  2.91916408
   0.8764552   0.04106978  0.3290891   0.08690082  2.98423589]
 [ 0.11810721  0.09600584  0.03439717  0.03531754  4.6109894   0.65869723
   1.19552989  0.68277012  0.10922341  2.39556682  0.02920874]
 [ 2.98329992  1.95337449  2.53776841 -0.54449361 -0.08221392 -0.02968081
  -0.04477322  2.7775048   3.00320418 -0.01376337  0.57133206]
 [ 4.81169574  3.04847219  3.45944129  0.39505758  0.12298993 -0.32749523
  -0.14407572  4.56787778  5.0775274   0.06774477 -0.18534872]
 [-0.0949532  -0.03982373 -0.15421435 -0.15172068  3.87822861  0.35813849
   0.941865    0.39195713 -0.10928442  1.9968667  -0.25792377]
 [ 4.18555696  2.69990683  3.4146455  -0.01892709 -0.06041327  0.22929478
   0.02905504  3.9208455   4.28917781  0.0259856   0.92850978]
 [-0.0524567  -0.17420892  0.14794935  4.

In [20]:
Sigma

array([14.2701248 , 11.19808631,  7.13480024,  5.13612006,  4.68588496,
        3.09682859,  2.72917436,  2.55571761,  1.05782196,  0.185364  ])

In [21]:
Sig4 = np.mat(np.eye(4)*Sigma[:4])
Sig4.I

matrix([[0.07007647, 0.        , 0.        , 0.        ],
        [0.        , 0.08930097, 0.        , 0.        ],
        [0.        , 0.        , 0.14015809, 0.        ],
        [0.        , 0.        , 0.        , 0.1946995 ]])

matrix([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [24]:
myMat

matrix([[0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 5],
        [0, 0, 0, 5, 0, 3, 0, 0, 0, 0, 3],
        [0, 0, 0, 0, 4, 1, 0, 1, 0, 4, 0],
        [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
        [5, 4, 2, 0, 0, 0, 0, 5, 5, 0, 0],
        [0, 0, 0, 0, 5, 0, 1, 0, 0, 0, 0],
        [4, 1, 4, 0, 0, 0, 0, 4, 5, 0, 1],
        [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
        [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
        [1, 0, 0, 4, 0, 0, 0, 1, 2, 0, 0]])

In [25]:
myMat.A

array([[0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 5],
       [0, 0, 0, 5, 0, 3, 0, 0, 0, 0, 3],
       [0, 0, 0, 0, 4, 1, 0, 1, 0, 4, 0],
       [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
       [5, 4, 2, 0, 0, 0, 0, 5, 5, 0, 0],
       [0, 0, 0, 0, 5, 0, 1, 0, 0, 0, 0],
       [4, 1, 4, 0, 0, 0, 0, 4, 5, 0, 1],
       [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
       [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
       [1, 0, 0, 4, 0, 0, 0, 1, 2, 0, 0]])

In [29]:
dataMat = myMat
user  = 1

U,Sigma,VT = np.linalg.svd(dataMat)
# 使用奇异值构建一个对角矩阵
Sig4 = np.mat(np.eye(4)*Sigma[:4])
# 利用U矩阵将物品转换到低维空间中
xformedItems = dataMat.T * U[:, :4] * Sig4.I
print('xformedItems=',xformedItems)
print('xformedItems行和列数', np.shape(xformedItems))

unratedItems = np.nonzero(dataMat[user, :].A == 0)[1]
print('dataMat[user, :].A=',dataMat[user, :].A)
print('np.nonzero(dataMat[user, :].A == 0)结果为', np.nonzero(dataMat[user, :].A == 0))

xformedItems= [[-4.94993155e-01 -8.55622175e-02 -2.48611806e-02  3.33301232e-02]
 [-3.13458547e-01 -7.52514984e-02 -7.48135856e-03 -3.50541410e-02]
 [-3.78450772e-01 -4.30110266e-02 -2.73248525e-02 -3.16127753e-01]
 [-9.66651163e-02  5.96631453e-01 -1.11181970e-01  7.08108827e-01]
 [-1.48606103e-02  4.20778007e-02  8.58513789e-01  4.28827005e-02]
 [-5.34500615e-02  4.98695994e-01  4.12721285e-02 -1.51444295e-01]
 [-1.53015570e-02  1.73637176e-01  1.96700179e-01 -5.96048540e-02]
 [-4.69111042e-01 -7.14523528e-02  8.17335025e-02  6.74284466e-02]
 [-5.18848182e-01 -6.64352384e-02 -3.46135811e-02  1.62309665e-01]
 [-1.56328052e-02  6.08803417e-02  4.39273573e-01  4.45686018e-04]
 [-1.06449559e-01  5.79097298e-01 -8.90946777e-02 -5.80592096e-01]]
xformedItems行和列数 (11, 4)
dataMat[user, :].A= [[0 0 0 5 0 3 0 0 0 0 3]]
np.nonzero(dataMat[user, :].A == 0)结果为 (array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 1, 2, 4, 6, 7, 8, 9]))


In [30]:
unratedItems

array([0, 1, 2, 4, 6, 7, 8, 9])

In [34]:
dataMat[user, :] == 0

matrix([[ True,  True,  True, False,  True, False,  True,  True,  True,
          True, False]])

In [38]:
np.nonzero((dataMat[user, :].A == 0))[1]

array([0, 1, 2, 4, 6, 7, 8, 9])

In [39]:
np.nonzero((dataMat[user, :].A == 0))

(array([0, 0, 0, 0, 0, 0, 0, 0]), array([0, 1, 2, 4, 6, 7, 8, 9]))

array([[ True,  True,  True, False,  True, False,  True,  True,  True,
         True, False]])